#### importing

In [1]:
import nltk 
import numpy as np 
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer
import json 
import pickle
lem=PorterStemmer()
import random
import math


#### intializing the necessary objects

In [2]:
words=[]   ## for lemmatized words
classes=[]
documents=[]  ## pattern with responses
ignore_words=["?",",","!","."]
df=open("intents.json")
intents=json.load(df)



#### testing (for me)

In [3]:
##for intent in intents['intents']:
    ##print (intent)

#### tokenization

In [4]:
for intent in intents['intents']:
    for pattern in intent["patterns"]:
        w=nltk.word_tokenize(pattern)
        ##print("tokenized word is: {}".format(w)) ## just ensurance
        words.extend(w)     ## take all the iterables and extend them in the []
        documents.append((w,intent['tag'])) ## appending the tokenized sentence with its class
        ## adding the tag to classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
    ## final words list
    ##print("words list is : {}".format(words)) ## this is for ensurance (for me)
    #print("docs is : {}".format(documents))
    #print("classes is : {}".format(classes))

#### stemming and lowering

In [5]:
words=[lem.stem(w.lower())for w in words if w not in ignore_words] ## lowering and lemmatization
words = list(set(words)) ## removing duplicates
classes = list(set(classes))
##print(words)
pickle.dump(words,open('words.pkl','wb'))  ## saving into a pickle file
pickle.dump(classes,open('classes.pkl','wb'))

In [6]:
training= []             ## we will have then bow 
output= [0]*len(classes) ## storing it with zero along with the length of the classes we have 

for doc in documents:
    bag=[]
    pattern_words=doc[0] ## (['Hi','there'],'greetings') 0 index is for the pattern itself
    pattern_words=[lem.stem(word.lower())for word in pattern_words] ##lemmatizing the patterns
    ##print("pattern words is : {}".format(pattern_words))
    for w in words:
        bag.append(1)if w in pattern_words else bag.append(0)
    ##print("bag is :{}".format(bag))
    output_row=list(output)
    output_row[classes.index(doc[1])]=1
   ## print("current output :{}".format(output_row)) ## the output of the class
     
     
    training.append([bag,output_row])
##print("training data is:{}".format(training))
random.shuffle(training) ## avoids the overfit
training=np.array(training)
train_x=list(training[:,0]) ##bag
train_y=list(training[:,1]) ## outputrow which has the classes
##print("x:{}".format(train_x))
##print("y:{}".format(train_y))
   
    




C:\Users\noor waleed\AppData\Local\Temp\ipykernel_20196\3393272716.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


### Creating a model with a simple seq model


In [7]:
from keras.models import Sequential
from keras.layers import Dense 
from keras.layers import Dropout


In [8]:
model= Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation="relu")) ## 128 neurons with number of input layer is the len of x
model.add(Dropout(0.5)) ## activates a certain neurons for a better learn and avoiding the overfitting
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))




#### compile the model and define an optomization fuction with stochastic gradient decent


In [9]:
from keras import optimizers
sgd=optimizers.SGD(learning_rate=0.003,decay=1e-6,momentum=0.9,nesterov=True) ## decay reduces the Lr the momentum and nesterov speed up the training

In [10]:
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [11]:
mfit=model.fit(np.array(train_x),np.array(train_y),epochs=199,batch_size=5,verbose=1) ## batch size is how many times do we change the parameters of our model
model.save("chatbot_model.h5",mfit)


Epoch 1/199
7/7 [==============================] - 3s 3ms/step - loss: 2.3240 - accuracy: 0.0588
Epoch 2/199
7/7 [==============================] - 0s 3ms/step - loss: 2.3935 - accuracy: 0.1471
Epoch 3/199
7/7 [==============================] - 0s 3ms/step - loss: 2.3172 - accuracy: 0.1176
Epoch 4/199
7/7 [==============================] - 0s 3ms/step - loss: 2.2699 - accuracy: 0.1176
Epoch 5/199
7/7 [==============================] - 0s 3ms/step - loss: 2.2376 - accuracy: 0.2059
Epoch 6/199
7/7 [==============================] - 0s 3ms/step - loss: 2.2824 - accuracy: 0.1471
Epoch 7/199
7/7 [==============================] - 0s 3ms/step - loss: 2.1827 - accuracy: 0.2059
Epoch 8/199
7/7 [==============================] - 0s 3ms/step - loss: 2.1091 - accuracy: 0.2353
Epoch 9/199
7/7 [==============================] - 0s 3ms/step - loss: 2.1079 - accuracy: 0.4118
Epoch 10/199
7/7 [==============================] - 0s 3ms/step - loss: 2.0542 - accuracy: 0.3529
Epoch 11/199
7/7 [===========